# 触ってみよう♪ ChainerCV 

# chainercv.transforms

# import modules

In [ ]:
from collections import OrderedDict
import itertools

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import chainer
import chainercv
from chainercv.visualizations import vis_image, vis_point

import PIL

# confirm chainer version

## chainer

In [ ]:
chainer.__version__

## chainercv

In [ ]:
chainercv.__version__

In [ ]:
imgfile='lena.jpeg'

# read image

In [ ]:
img = chainercv.utils.read_image(imgfile)
img.shape  # should be C, H, W

# visualize what you read

In [ ]:
chainercv.visualizations.vis_image(img)

In [ ]:
# https: // github.com/matplotlib/matplotlib/issues/7221/
from mpl_toolkits.axes_grid1 import make_axes_locatable

## visualize image on figure which user defined

In [ ]:
img = chainercv.utils.read_image(imgfile)
fig = plt.figure(figsize=(10, 5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax2.hist([img[i].flatten() for i in range(3)],
         bins=range(0, 256+1, 8), color=['r', 'g', 'b'])
ax1.set_title('chainercv.vis_image')
ax2.set_title('color histgram')
vis_image(img, ax=ax1)

In [ ]:
from chainercv.visualizations import vis_image
import chainercv.transforms as transforms

# `chainer.transforms` exercise

## center_crop 
 - `chainercv.transforms.center_crop(img, size, return_param=False, copy=False)`

In [ ]:
h = 150
w = 120
size = (h, w)
vis_image(transforms.center_crop(img, size))

### return_param=True (default False)

In [ ]:
cropped, param = transforms.center_crop(img, size, return_param=True)
param

### difference between copy=False and copy=True (default False)

- what happens if we modify data of  `cropped=transforms.center_crop(img,size,copy=False)` ?

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))
img = chainercv.utils.read_image(imgfile)
cropped = transforms.center_crop(img, size, copy=False)
# modify data of original image
img[:, param['y_slice'], param['x_slice']] = 0  # fill with black
vis_image(cropped, ax=ax1)
# modify data of cropped image
cropped[:, :, :] = 128  # fill with gray
vis_image(img, ax=ax2)
vis_image(cropped, ax=ax3)

# check id of view

In [ ]:
print('object id', id(img), id(cropped), id(img) == id(cropped))
print('view id', id(img.view) == id(cropped.view)) # should be `True`

### avoid side effect

- we can avoid such a side effect by setting optional argument `copy=True`

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))
img = chainercv.utils.read_image(imgfile)
cropped = transforms.center_crop(img, size, copy=True)
# modify data of original image
img[:, param['y_slice'], param['x_slice']] = 0  # fill with black
vis_image(cropped, ax=ax1)
# modify data of cropped image
cropped[:, :, :] = 128  # fill with gray
vis_image(img, ax=ax2)
vis_image(cropped, ax=ax3)

## flip 
 - `chainercv.transforms.flip(img, y_flip=False, x_flip=False, copy=False)`

In [ ]:
img = chainercv.utils.read_image(imgfile)
f, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=1, ncols=4, figsize=(13, 5))
img_y_flip = transforms.flip(img, y_flip=True, x_flip=False)
ax2.set_title('y_flip=True')
vis_image(img_y_flip, ax=ax2)
img_x_flip = transforms.flip(img, y_flip=False, x_flip=True)
ax3.set_title('x_flip=True')
vis_image(img_x_flip, ax=ax3)
img_xy_flip = transforms.flip(img, y_flip=True, x_flip=True)
ax4.set_title('x_flip=True, y_flip=True')
vis_image(img_xy_flip, ax=ax4)
# do nothing
default = transforms.flip(img, y_flip=False, x_flip=False)
ax1.set_title('default')
vis_image(default, ax=ax1)

### check side effect (copy = False)

In [ ]:
img = chainercv.utils.read_image(imgfile)
f, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=1, ncols=4, figsize=(13, 5))
img_y_flip = transforms.flip(img, y_flip=True, copy=False)
img_xy_flip = transforms.flip(img_y_flip, x_flip=True, copy=False)
ax1.set_title('original')
vis_image(img, ax=ax1)
ax2.set_title('xy_flip')
vis_image(img_xy_flip, ax=ax2)
img[:, 30:50, 20:50] = 128
img[:, 150:180, 100:200] = 0
ax3.set_title("modified")
vis_image(img, ax=ax3)
ax4.set_title("modified xy_flip")
vis_image(img_xy_flip, ax=ax4)

## pca_lighting

- `chainercv.transforms.pca_lighting(img, sigma, eigen_value=None, eigen_vector=None)`

In [ ]:
img = chainercv.utils.read_image(imgfile)
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
vis_image(img, ax=ax1)
vis_image(transforms.pca_lighting(img, sigma=25.5), ax=ax2)

## try pca_lighting several times

In [ ]:
for _ in range(5):
    vis_image(transforms.pca_lighting(img, sigma=25.5))

## can we apply to grayscale ?
- no ...

In [ ]:
#gray = chainercv.utils.read_image('lena.jpeg',color=False)
#transforms.pca_lighting(gray,sigma=25)

# `chainercv.transforms` exer with random

## transforms.random_crop
- chainercv.transforms.random_crop(img, size, return_param=False, copy=False)

In [ ]:
img = chainercv.utils.read_image(imgfile)
f, axes = plt.subplots(nrows=1, ncols=5, figsize=(15, 5))

h = 120
w = 100
size = (h, w)
for i in range(5):
    cropped = transforms.random_crop(img, size)
    vis_image(cropped, ax=axes[i])

## transforms.random_expand
- chainercv.transforms.random_expand(img, max_ratio=4, fill=0, return_param=False)

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=5, figsize=(15, 5))

h = 120
w = 100
size = (h, w)
for i in range(5):
    fill = np.random.randint(0, 255+1, size=3)
    cropped = transforms.random_expand(img, max_ratio=1.7, fill=fill)
    vis_image(cropped, ax=axes[i])

## transforms.random_flip
- transforms.random_flip(img, y_random=False, x_random=False, return_param=False, copy=False)

In [ ]:
img = chainercv.utils.read_image(imgfile)
nrows = 2
ncols = 4
f, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 8))
for (r, c) in itertools.product(range(nrows), range(ncols)):
    img_flipped, param = transforms.random_flip(
        img, y_random=True, x_random=True, return_param=True)
    ax = axes[r, c]
    ax.set_title(' '.join(['='.join([str(k), str(v)])
                           for k, v in param.items()]))
    vis_image(img_flipped, ax=ax)

# transforms.random_rotate

- chainercv.transforms.random_rotate(img, return_param=False)

In [ ]:
img = chainercv.utils.read_image(imgfile)
nrows = 2
ncols = 4
f, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 8))
for (r, c) in itertools.product(range(nrows), range(ncols)):
    img_rot, param = transforms.random_rotate(img, return_param=True)
    ax = axes[r, c]
    ax.set_title(' '.join(['='.join([str(k), str(v)])
                           for k, v in param.items()]))
    vis_image(img_rot, ax=ax)

## transforms.random_sized_crop

- chainercv.transforms.random_sized_crop(img, scale_ratio_range=(0.08, 1), aspect_ratio_range=(0.75, 1.3333333333333333), return_param=False, copy=False)

In [ ]:
img = chainercv.utils.read_image(imgfile)
nrows = 2
ncols = 4
f, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 8))
for (r, c) in itertools.product(range(nrows), range(ncols)):
    img_cropped, param = transforms.random_sized_crop(img, return_param=True)
    ax = axes[r, c]
    vis_image(img_cropped, ax=ax)

# resize exercise

## chainercv.transforms.resize
- chainercv.transforms.resize(img, size, interpolation=2)

- where the value of `interpolation` is method of resize algorithm

```python
import numpy as np
import PIL
import warnings


try:
    import cv2

    def _resize(img, size, interpolation):
        img = img.transpose((1, 2, 0))
        if interpolation == PIL.Image.NEAREST:
            cv_interpolation = cv2.INTER_NEAREST
        elif interpolation == PIL.Image.BILINEAR:
            cv_interpolation = cv2.INTER_LINEAR
        elif interpolation == PIL.Image.BICUBIC:
            cv_interpolation = cv2.INTER_CUBIC
        elif interpolation == PIL.Image.LANCZOS:
            cv_interpolation = cv2.INTER_LANCZOS4
        H, W = size
        img = cv2.resize(img, dsize=(W, H), interpolation=cv_interpolation)

        # If input is a grayscale image, cv2 returns a two-dimentional array.
        if len(img.shape) == 2:
            img = img[:, :, np.newaxis]
        return img.transpose((2, 0, 1))

except ImportError:
    def _resize(img, size, interpolation):
        warnings.warn(
            'cv2 is not installed on your environment. '
            'ChainerCV will fall back on Pillow. '
            'Installation of cv2 is recommended for faster computation. ',
            RuntimeWarning)

        C = img.shape[0]
        H, W = size
        out = np.empty((C, H, W), dtype=img.dtype)
        for ch, out_ch in zip(img, out):
            ch = PIL.Image.fromarray(ch, mode='F')
            out_ch[:] = ch.resize((W, H), resample=interpolation)
        return out


def resize(img, size, interpolation=PIL.Image.BILINEAR):
    """Resize image to match the given shape.

    This method uses :mod:`cv2` or :mod:`PIL` for the backend.
    If :mod:`cv2` is installed, this function uses the implementation in
    :mod:`cv2`. This implementation is faster than the implementation in
    :mod:`PIL`. Under Anaconda environment,
    :mod:`cv2` can be installed by the following command.

    .. code::

        $ conda install -c menpo opencv3=3.2.0

    Args:
        img (~numpy.ndarray): An array to be transformed.
            This is in CHW format and the type should be :obj:`numpy.float32`.
        size (tuple): This is a tuple of length 2. Its elements are
            ordered as (height, width).
        interpolation (int): Determines sampling strategy. This is one of
            :obj:`PIL.Image.NEAREST`, :obj:`PIL.Image.BILINEAR`,
            :obj:`PIL.Image.BICUBIC`, :obj:`PIL.Image.LANCZOS`.
            Bilinear interpolation is the default strategy.

    Returns:
        ~numpy.ndarray: A resize array in CHW format.

    """
    img = _resize(img, size, interpolation)
    return img
```

### resize size/2

In [ ]:
img = chainercv.utils.read_image(imgfile)
nrows = 2
ncols = 2
f, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 12))
size = (img.shape[1]//2, img.shape[2]//2)
algorithms = ['NEAREST', 'BILINEAR', 'BICUBIC', 'LANCZOS']
for idx, algo in enumerate(algorithms):
    q, r = divmod(idx, 2)
    ax = axes[q, r]
    ax.set_title(algorithms[idx])
    resized = transforms.resize(img, size, eval('PIL.Image.{}'.format(algo)))
    vis_image(resized, ax=ax)

### resize 2*size

In [ ]:
img = chainercv.utils.read_image(imgfile)
nrows = 2
ncols = 2
f, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 12))
size = (img.shape[1]*2, img.shape[2]*2)
for idx, interpolation in enumerate([PIL.Image.NEAREST, PIL.Image.BILINEAR, PIL.Image.BICUBIC, PIL.Image.LANCZOS]):
    q, r = divmod(idx, 2)
    ax = axes[q, r]
    resized = transforms.resize(img, size, interpolation)
    vis_image(resized, ax=ax)

## transforms.resize_contain
- `transforms.resize_contain(img, size, fill=0, interpolation=2, return_param=False)`

In [ ]:
img = chainercv.utils.read_image(imgfile)
f, ax = plt.subplots(figsize=(12, 12))
h = 120
w = 60
size = (h, w)
resized, _param = transforms.resize_contain(img, size, return_param=True)
param = OrderedDict()
for k in sorted(list(_param.keys())):
    param[k] = _param[k]
ax.set_title(' '.join(['='.join([str(k), str(v)])for k, v in param.items()]))
pts = [
    [param['y_offset'], param['x_offset']],
    [param['y_offset']+param['scaled_size'][0], param['x_offset']],
    [param['y_offset'], param['x_offset']+param['scaled_size'][1]],
    [param['y_offset']+param['scaled_size'][0],
        param['x_offset']+param['scaled_size'][1]],
]
vis_point(resized, pts, ax=ax)
vis_image(resized, ax=ax)

In [ ]:
img = chainercv.utils.read_image(imgfile)
f, ax = plt.subplots(figsize=(12, 12))
h = 256
w = 512
size = (h, w)
resized, _param = transforms.resize_contain(img, size, return_param=True)
param = OrderedDict()
for k in sorted(list(_param.keys())):
    param[k] = _param[k]
ax.set_title(' '.join(['='.join([str(k), str(v)])for k, v in param.items()]))
pts = [
    [param['y_offset'], param['x_offset']],
    [param['y_offset']+param['scaled_size'][0], param['x_offset']],
    [param['y_offset'], param['x_offset']+param['scaled_size'][1]],
    [param['y_offset']+param['scaled_size'][0],
        param['x_offset']+param['scaled_size'][1]],
]
vis_point(resized, pts, ax=ax)
vis_image(resized, ax=ax)

# transforms.rotate
- `transforms.rotate(img, angle, expand=True)`

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))
img = chainercv.utils.read_image(imgfile)
degree = 30
img_rot_expand = transforms.rotate(img, degree, expand=True)
img_rot_expand_false = transforms.rotate(img, degree, expand=False)
ax1.set_title('original')
ax2.set_title('expand True')
ax3.set_title('expand False')
vis_image(img, ax=ax1)
vis_image(img_rot_expand, ax=ax2)
vis_image(img_rot_expand_false, ax=ax3)

# transforms.scale
- `transforms.scale(img, size, fit_short=True, interpolation=2)`

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=(5, 10))
img = chainercv.utils.read_image(imgfile)
img = transforms.resize(img, (128, 256))
img_fit_short = transforms.scale(img, size=80, fit_short=True)
img_fit_short_false = transforms.scale(img, size=80, fit_short=False)
ax1.set_title('original')
ax2.set_title('fit_shot True')
ax3.set_title('fit_shot False')
ax1.set_title('img.shape[1:]={}'.format(img.shape[1:]))
ax2.set_title('fit_short={}, img.shape[1:]={}'.format(
    True, img_fit_short.shape[1:]))
ax3.set_title('fit_short={}, img.shape[1:]={}'.format(
    False, img_fit_short_false.shape[1:]))
vis_image(img, ax=ax1)
vis_image(img_fit_short, ax=ax2)
vis_image(img_fit_short_false, ax=ax3)

# transforms.ten_crop
- `transforms.ten_crop(img, size)`

Crop 10 regions from an array.

This method crops 10 regions. All regions will be in shape `size`. These regions consist of 1 center crop and 4 corner crops and horizontal flips of them.

The crops are ordered in this order.

- center crop
- top-left crop
- bottom-left crop
- top-right crop
- bottom-right crop
- center crop (flipped horizontally)
- top-left crop (flipped horizontally)
- bottom-left crop (flipped horizontally)
- top-right crop (flipped horizontally)
- bottom-right crop (flipped horizontally)

In [ ]:
f, axes = plt.subplots(nrows=3, ncols=3, figsize=(10, 10))
img = chainercv.utils.read_image(imgfile)
ten_crop = transforms.ten_crop(img, size=(80, 80))

positions = [
    (1, 1),  # center crop
    (0, 0),  # top-left crop
    (2, 0),  # bottom-left crop
    (0, 2),  # top-right crop
    (2, 2),  # bottom-right crop
]

for (r, c) in itertools.product(range(3), repeat=2):
    if not (r, c) in positions:
        axes[r, c].axis('off')

for (r, c), i in zip(positions, range(5)):
    vis_image(ten_crop[i], ax=axes[r, c])

In [ ]:
f, axes = plt.subplots(nrows=3, ncols=3, figsize=(10, 10))
img = chainercv.utils.read_image(imgfile)
ten_crop = transforms.ten_crop(img, size=(80, 80))

positions = [
    (1, 1),  # center crop
    (0, 0),  # top-left crop
    (2, 0),  # bottom-left crop
    (0, 2),  # top-right crop
    (2, 2),  # bottom-right crop
]

for (r, c) in itertools.product(range(3), repeat=2):
    if not (r, c) in positions:
        axes[r, c].axis('off')

for (r, c), i in zip(positions, range(5)):
    vis_image(ten_crop[5+i], ax=axes[r, c])

# What I do next ...
- study bbox etc...

# Reference:

- https://chainercv.readthedocs.io/en/stable/reference/transforms.html